<a href="https://colab.research.google.com/github/abhiramraikal07/Heap-Management-System/blob/main/NFT_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Networks
#Dog Breed Identification(NFT Project)

---

* [Step 1](#step1): Import Datasets
* [Step 2](#step2): Detect Dogs
* [Step 3](#step3): Create a CNN to Classify Dog Breeds (from Scratch)
* [Step 4](#step4): Create a CNN to Classify Dog Breeds (using Transfer Learning)
* [Step 5](#step5): Write your Algorithm
* [Step 6](#step6): Test Your Algorithm

---
<a id='step0'></a>
## Step 1: Import Datasets

* Download the [dog dataset](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip).  Unzip the folder and place it in this project's home directory, at the location `/dog_images`. 


In [ ]:
%%time
# https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
# https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/lfw.zip

import requests, zipfile, io, os, shutil

root = '/content'
dogimages_url = "https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip"
humanimages_url = "https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/lfw.zip"

data_path = os.path.join(root,'data')
dogimages_path = os.path.join(data_path,'dogImages')
humanimages_path = os.path.join(data_path,'lfw')
data_path = os.path.join(root,'data')
if not os.path.exists(data_path): os.mkdir(data_path)

# if os.path.exists(dogimages_path):shutil.rmtree(dogimages_path)
# if os.path.exists(humanimages_path):shutil.rmtree(humanimages_path)

if not os.path.exists(dogimages_path):
  print("downloading dog images")
  r = requests.get(dogimages_url) 
  z = zipfile.ZipFile(io.BytesIO(r.content)) 
  z.extractall(data_path)

if not os.path.exists(humanimages_path):
  print("downloading human images")
  r = requests.get(humanimages_url) 
  z = zipfile.ZipFile(io.BytesIO(r.content)) 
  z.extractall(data_path)

import numpy as np
from glob import glob

# load filenames for human and dog images
human_files = np.array(glob("/content/data/lfw/*/*"))
dog_files = np.array(glob("/content/data/dogImages/*/*/*"))

# print number of images in each dataset
print('There are %d total human images.' % len(human_files))
print('There are %d total dog images.' % len(dog_files))

There are 13233 total human images.
There are 8351 total dog images.
CPU times: user 114 ms, sys: 67.1 ms, total: 181 ms
Wall time: 183 ms


In [ ]:
if not os.path.exists('/content/haarcascades'): os.mkdir('/content/haarcascades')
haarcascades_url = 'https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_alt.xml'
!wget -O /content/haarcascades/haarcascade_frontalface_alt.xml {haarcascades_url}

--2019-12-25 23:23:38--  https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_alt.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 676709 (661K) [text/plain]
Saving to: ‘/content/haarcascades/haarcascade_frontalface_alt.xml’

/content/haarcascad 100%[===================>] 660.85K  --.-KB/s    in 0.04s   

2019-12-25 23:23:39 (14.8 MB/s) - ‘/content/haarcascades/haarcascade_frontalface_alt.xml’ saved [676709/676709]



---
<a id='step2'></a>
## Step 2: Detect Dogs

In [ ]:
import torch
import torchvision.models as models

# define VGG16 model
VGG16 = models.vgg16(pretrained=True)

# check if CUDA is available
use_cuda = torch.cuda.is_available()

# move model to GPU if CUDA is available
if use_cuda:
    VGG16 = VGG16.cuda()

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:11<00:00, 49.2MB/s]


### (IMPLEMENTATION) Making Predictions with a Pre-trained Model



In [ ]:
from PIL import Image
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets
import os

def VGG16_predict(img_path):
    '''
    Use pre-trained VGG-16 model to obtain index corresponding to 
    predicted ImageNet class for image at specified path
    
    Args:
        img_path: path to an image
        
    Returns:
        Index corresponding to VGG-16 model's prediction
    '''
    
    ## TODO: Complete the function.
    ## Load and pre-process an image from the given img_path
    ## Return the *index* of the predicted class for that image
    
    # data transformation
    batch_size = 64
    img = Image.open(img_path).convert('RGB')
    image_transforms = transforms.Compose([
                            transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
                            transforms.RandomRotation(degrees=15),
                            transforms.ColorJitter(),
                            transforms.RandomHorizontalFlip(),
                            transforms.CenterCrop(size=224),  # Image net standards
                            transforms.ToTensor(),
                            transforms.Normalize([0.485, 0.456, 0.406],
                                                 [0.229, 0.224, 0.225])  # Imagenet standards
                        ])
    
    image = image_transforms(img)[:3,:,:].unsqueeze(0)
#     image = image_transforms(img)
#     print(image_transformation)
    
    if use_cuda:
        image = image.cuda()
    output = VGG16(image)

    _,pred = torch.max(output, dim=1)
    pred=pred.cpu()
    pred = pred.data.numpy()[0]
        
    return pred # predicted class index

VGG16_predict(dog_files[0])


180

### (IMPLEMENTATION) Write a Dog Detector



In [ ]:
### returns "True" if a dog is detected in the image stored at img_path
def dog_detector(img_path):
    ## TODO: Complete the function.
#     in VGG16 index 151 to 268 are dog classifications
    
    return VGG16_predict(img_path)>= 151 and VGG16_predict(img_path)<=268 # true/false
print(dog_detector(human_files[3]))
print(dog_detector(dog_files[7]))

False
True


### (IMPLEMENTATION) Assess the Dog Detector



In [ ]:
### TODO: Test the performance of the dog_detector function
### on the images in human_files_short and dog_files_short. 

human_files_count=0
dog_files_count=0
for i in range(len(human_files_short)):
    if dog_detector(human_files_short[i]):
        human_files_count+=1
    if dog_detector(dog_files_short[i]):
        dog_files_count+=1
print("number of dog faces detected in human_fiiles_short {0}".format(human_files_count))
print("number of dog faces detected in dog_files_short {0}".format(dog_files_count))


number of dog faces detected in human_fiiles_short 5
number of dog faces detected in dog_files_short 99


---
<a id='step3'></a>
## Step 3: Create a CNN to Classify Dog Breeds (from Scratch)

### (IMPLEMENTATION) Specify Data Loaders for the Dog Dataset



In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_path = '/content/data/dogImages/train/'
val_path = '/content/data/dogImages/valid'
test_path = '/content/data/dogImages/test'

batch_size=64
train_generator = train_datagen.flow_from_directory(
        train_path,
        # '/content/cats_dogs/data/train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical') 

validation_generator = val_datagen.flow_from_directory(
        val_path,
        # '/content/cats_dogs/data/val',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')

test_generator = val_datagen.flow_from_directory(
        test_path,
        # '/content/cats_dogs/data/val',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')



Found 6680 images belonging to 133 classes.
Found 835 images belonging to 133 classes.
Found 836 images belonging to 133 classes.


### (IMPLEMENTATION) Model Architecture


In [ ]:
from time import time
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=( 150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


# model.add(Conv2D(64, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten()) 
model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(133))
model.add(Activation('softmax'))

### (IMPLEMENTATION) Specify Loss Function and Optimizer



In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

### (IMPLEMENTATION) Train and Validate the Model


In [ ]:
start_time = time()
model.fit_generator(
        train_generator,
        #steps_per_epoch=18631 // batch_size,
        epochs=16,
        validation_data=validation_generator,
        #validation_steps=10119 // batch_size
        )
model.save_weights('first_try.h5')

print('time taken ',time()-start_time)

Epoch 1/16
105/105 [==============================] - 92s 875ms/step - loss: 4.8920 - acc: 0.0086 - val_loss: 4.8711 - val_acc: 0.0156
Epoch 2/16
105/105 [==============================] - 91s 863ms/step - loss: 4.7967 - acc: 0.0196 - val_loss: 4.6907 - val_acc: 0.0299
Epoch 3/16
105/105 [==============================] - 91s 868ms/step - loss: 4.5571 - acc: 0.0423 - val_loss: 4.4803 - val_acc: 0.0455
Epoch 4/16
105/105 [==============================] - 91s 865ms/step - loss: 4.3324 - acc: 0.0559 - val_loss: 4.3741 - val_acc: 0.0527
Epoch 5/16
105/105 [==============================] - 91s 864ms/step - loss: 4.1674 - acc: 0.0750 - val_loss: 4.2057 - val_acc: 0.0671
Epoch 6/16
105/105 [==============================] - 90s 857ms/step - loss: 4.0481 - acc: 0.0872 - val_loss: 4.2626 - val_acc: 0.0743
Epoch 7/16
105/105 [==============================] - 91s 866ms/step - loss: 3.9246 - acc: 0.1028 - val_loss: 4.1273 - val_acc: 0.0766
Epoch 8/16
105/105 [==============================] - 9

### (IMPLEMENTATION) Test the Model



In [ ]:
# test_prediction = model.predict(test_generator)
model.metrics_names

['loss', 'acc']

In [ ]:
# from sklearn.metrics import accuracy_score
# model.evaluate_generator(test_generator)
test_generator.reset()
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_eval = model.evaluate_generator(test_generator,STEP_SIZE_TEST)
print('test loss ',test_eval[0])
print('test accuracy ',test_eval[1])

test loss  4.124664765137893
test accuracy  0.125


---
<a id='step4'></a>
## Step 4: Create a CNN to Classify Dog Breeds (Optimization)

### (IMPLEMENTATION) Specify Data Loaders for the Dog Dataset


In [ ]:
## TODO: Specify data loaders
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_path = '/content/data/dogImages/train/'
val_path = '/content/data/dogImages/valid'
test_path = '/content/data/dogImages/test'

batch_size=64
train_generator = train_datagen.flow_from_directory(
        train_path,
        # '/content/cats_dogs/data/train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical') 

validation_generator = val_datagen.flow_from_directory(
        val_path,
        # '/content/cats_dogs/data/val',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')

test_generator = val_datagen.flow_from_directory(
        test_path,
        # '/content/cats_dogs/data/val',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')

Using TensorFlow backend.


Found 6680 images belonging to 133 classes.
Found 835 images belonging to 133 classes.
Found 836 images belonging to 133 classes.


### (IMPLEMENTATION) Model Architecture


In [ ]:
from time import time
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from keras.applications.vgg16 import VGG16 as PTModel, preprocess_input
import keras
keras.backend.set_learning_phase(1)

img_rows, img_cols, img_channel = 150, 150, 3
base_model = PTModel(weights='imagenet'
                     ,include_top=False, input_shape=(img_rows, img_cols, img_channel), classes = 2)
add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(64, activation='relu'))
add_model.add(Dense(133, activation='sigmoid'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))

for layer in base_model.layers:
    layer.trainable = False

    if layer.name.startswith('bn'):
        layer.call(layer.input, training=False)





58892288/58889256 [==============================] - 2s 0us/step








### (IMPLEMENTATION) Specify Loss Function and Optimizer



In [ ]:
model.compile(loss='binary_crossentropy', 
              optimizer=SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


### (IMPLEMENTATION) Train and Validate the Model


In [ ]:
check_point_name = 'vgg16.model'
model_weights = 'vgg16.h5'

start_time = time()
model.fit_generator(
        train_generator,
        epochs=2,
        validation_data=validation_generator,
        #class_weight = class_weights,
        callbacks=[ModelCheckpoint(check_point_name, monitor='val_acc', save_best_only=True)])
model.save_weights(model_weights)

print('time taken ',time()-start_time)



Epoch 1/2
105/105 [==============================] - 93s 890ms/step - loss: 0.6722 - acc: 0.5667 - val_loss: 0.6494 - val_acc: 0.6067
Epoch 2/2
105/105 [==============================] - 90s 853ms/step - loss: 0.6352 - acc: 0.6307 - val_loss: 0.6239 - val_acc: 0.6455
time taken  183.86760330200195


### (IMPLEMENTATION) Test the Model



In [ ]:
# test(loaders_transfer, model_transfer, criterion_transfer, use_cuda)
test_generator.reset()
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_eval = model.evaluate_generator(test_generator,STEP_SIZE_TEST)
print('test loss ',test_eval[0])
print('test accuracy ',test_eval[1])

test loss  0.6086689554727994
test accuracy  0.6575513298694904


### (IMPLEMENTATION) Predict Dog Breed with the Model



In [ ]:
### TODO: Write a function that takes a path to an image as input
### and returns the dog breed that is predicted by the model.

# list of class names by index, i.e. a name can be accessed like class_names[0]
# class_names = [item[4:].replace("_", " ") for item in data_transfer['train'].classes]
from PIL import Image
import numpy as np
from skimage import transform

def load(filename):
   np_image = Image.open(filename)
   np_image = np.array(np_image).astype('float32')/255
   np_image = transform.resize(np_image, (150, 150, 3))
   np_image = np.expand_dims(np_image, axis=0)
   return np_image

def predict_breed_transfer(img_path):
    # load the image and return the predicted breed
  image = load(img_path)
  y_prob = model.predict(image)
  y_class = y_prob.argmax(axis=-1)
  # print(y_classes)
  labels = (train_generator.class_indices)
  labels = dict((v,k) for k,v in labels.items())
  prediction = [labels[k] for k in y_class]
  return prediction    

---
<a id='step5'></a>
## Step 5:  Algorithm


### Implementation of Algorithm

In [ ]:
### TODO: Write your algorithm.
### Feel free to use as many code cells as needed.

def run_app(img_path):
    ## handle cases for a human face, dog, and neither
    if haar_face_detector(img_path):
      print('hello human')
      prediction = predict_breed_transfer(img_path)
      print(prediction)
    elif dog_detector(img_path):
      print('hello dog')
      prediction = predict_breed_transfer(img_path)
      print(prediction)
    else:
      print("couldn't detect dog or human image")



---
<a id='step6'></a>
## Step 6: Testing the Algorithm

### (IMPLEMENTATION) Test the Algorithm on Sample Images!


In [ ]:
dog_files[:3]

array(['/content/data/dogImages/train/008.American_staffordshire_terrier/American_staffordshire_terrier_00547.jpg',
       '/content/data/dogImages/train/008.American_staffordshire_terrier/American_staffordshire_terrier_00616.jpg',
       '/content/data/dogImages/train/008.American_staffordshire_terrier/American_staffordshire_terrier_00612.jpg'],
      dtype='<U113')

In [ ]:
## TODO: Execute your algorithm from Step 6 on
## at least 6 images on your computer.
## Feel free to use as many code cells as needed.

## suggested code, below
for file in np.hstack((human_files[:3], dog_files[:3])):
    run_app(file)

hello human
['123.Pomeranian']
hello human
['013.Australian_terrier']
hello human
['123.Pomeranian']
hello dog
['123.Pomeranian']
hello human
['123.Pomeranian']
hello dog
['123.Pomeranian']
